In [2]:
# data handling
import csv
import pandas as pd
import numpy as np
import nvector as nv

# map creation
import folium
import folium.plugins as plugins
import ipywidgets as widgets

# GUI creation
from IPython.display import HTML
from IPython.display import display

# csv file -> pandas dataframe
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

# pandas dataframe -> python dictionary
def get_div_origin_dest(div_data_df):
    div_data = dict()
    for row in div_data_df.itertuples():
        if row.MAP_DATE not in div_data: # no map date
            #what on earth? setting a row of a dict to the dict itself?
            div_data[row.MAP_DATE] = dict()
            div_data[row.MAP_DATE]['POINT_Y'] = row.POINT_Y
            div_data[row.MAP_DATE]['POINT_X'] = row.POINT_X
    div_locs = []
    
    for key in sorted(div_data):
        div_loc = dict()
        div_loc['POINT_Y'] = div_data[key]['POINT_Y']
        div_loc['POINT_X'] = div_data[key]['POINT_X']
        div_locs.append(div_loc)
    
    #there is an origin and dest
    if len(div_locs) > 1:
        origin = div_locs[0]
        dest = div_locs[len(div_locs)-1]
        return [[origin['POINT_Y'],origin['POINT_X']],
                [dest['POINT_Y'],dest['POINT_X']]]
    # there isn't an origin / dest
    else:
        return None

# gets the vector's bearing from North
# why not just get the azimuth using other code?
def get_bearing(orig, dest):
    wgs84 = nv.FrameE(name='WGS84')
    pointA = wgs84.GeoPoint(latitude=orig[0], longitude=orig[1], degrees=True)
    pointB = wgs84.GeoPoint(latitude=dest[0], longitude=dest[1], degrees=True)
    p_AB_E = nv.diff_positions(pointA, pointB)
    frame_N = nv.FrameN(pointA)
    p_AB_N = p_AB_E.change_frame(frame_N)
    p_AB_N = p_AB_N.pvector.ravel()
    azimuth = np.arctan2(p_AB_N[1], p_AB_N[0])
    return np.rad2deg(azimuth)

# arrays for latitude, longitude for origin, destination
origin_lats = []
origin_lons = []
dest_lats = []
dest_lons = []

#iterate over divisions
for group_key in grouped_div.groups.keys():
    #weed out groups that don't have a valid num
    if isinstance(group_key[1],str): # if div num is a string
        div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','MAP_DATE']]
        #get division's origin and destination
        div_coords = get_div_origin_dest(div_df)
        if div_coords is not None: # if we already have coordinates
            origin_lats.append(div_coords[0][0])
            origin_lons.append(div_coords[0][1])
            dest_lats.append(div_coords[1][0])
            dest_lons.append(div_coords[1][1])

# Create NVector GeoPoint objects            
points = nv.GeoPoint(latitude=origin_lats,longitude=origin_lons,degrees=True)
nvectors = points.to_nvector()

# why do we keep doing conversion between NVector / pandas / python dict / etc.
# objects simply to get lat / long which we've already calculated?

n_EM_E = nvectors.mean_horizontal_position()
g_EM_E = n_EM_E.to_geo_point()
lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
mean_origin = [lat[0],lon[0]]

points = nv.GeoPoint(latitude=dest_lats,longitude=dest_lons,degrees=True)
nvectors = points.to_nvector()
n_EM_E = nvectors.mean_horizontal_position()
g_EM_E = n_EM_E.to_geo_point()
lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
mean_dest = [lat[0],lon[0]]

#now create the map
div_map = folium.Map([54.78, 32.04],zoom_start=6)

# Create and add circleMarker objects to map
folium.CircleMarker(mean_origin,radius=4,color='green').add_to(div_map)
folium.CircleMarker(mean_dest,radius=4,color='red').add_to(div_map)

# Create and add line (without arrow) to map
orig_dest_pair = [mean_origin,mean_dest]
vect_line = folium.PolyLine(locations=orig_dest_pair,weight=1,color='black')
div_map.add_child(vect_line)

# get vector direction
rotation = get_bearing(mean_origin,mean_dest) - 90 #to account for eastward initial rotation

# create points with given lat / long
arrow_lats = np.linspace(mean_origin[0], mean_dest[0], 4)[1:3]
arrow_lons = np.linspace(mean_origin[1], mean_dest[1], 4)[1:3]

# generate arrow (3-sided / triangle) objects at lat/long
for points in zip(arrow_lats, arrow_lons):
    folium.RegularPolygonMarker(location=points, 
                                fill_color='black', number_of_sides=3, 
                                radius=4, rotation=rotation).add_to(div_map)
# display map
div_map